In [1]:
# Packages
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

In [3]:
# Loading of the model
model_orig = read_sbml_model('iJN678mma.xml')
model = model_orig.copy()

In [4]:
# Calculate max biomass and squalene production
with model:
    base_max_growth = model.slim_optimize()
    print("Max biomass growth:", base_max_growth)
    model.objective = model.reactions.SQLS
    base_max_production = model.slim_optimize()
    print("Max squalene production:", base_max_production)

Max biomass growth: 0.06328458972007817
Max squalene production: 0.11155394524959755


In [5]:
model.solver = "glpk" # no idea

In [6]:
# Specify biomass growth rate
biomass = model.reactions.BIOMASS_Ec_SynAuto.lower_bound = 0.04859441594166733
# Specify model objective towards squalene biosynthesis
demand = model.reactions.get_by_id("SQLS")
model.objective = demand

In [10]:
# Max biomass and squalene production after objective change
with model:
    base_max_growth = model.slim_optimize()
    print("Max biomass growth:", base_max_growth)
    model.objective = model.reactions.SQLS
    base_max_production = model.slim_optimize()
    print("Max squalene production:", base_max_production)

Max biomass growth: 0.022158756023818647
Max squalene production: 0.022158756023818647


In [12]:
# State the product yield calculation
py = product_yield(model.reactions.SQLS, model.reactions.EX_co2_e)
#py = product_yield(model.reactions.SQLS, model.reactions.EX_glc__D_e)
#py = product_yield(model.reactions.SQLS, model.reactions.BIOMASS_Ec_SynAuto)

If needed, you can set different co-factor pairs by adding the parameter **cofactor_id_swaps=([from], [to])** which requires the metabolite identifiers in the model of the pairs that you want to swap.

Determine the co-factors in our model:

- NADH   = nadh_c
- NAD+   = nad_c
- NADPH  = nadph_c
- NADP+  = nadp_c

In [ ]:
# To swap NADPH to NADH, add "cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c'])" 
# inside the "CofactorSwapOptimization()"

#optswap = CofactorSwapOptimization(model=model, objective_function=py, plot=True, cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']))
optswap = CofactorSwapOptimization(model = model, objective_function = py)

In [ ]:
# Record possible co-factor swap targets and how long it takes to determine them
%%time
result = optswap.run(max_size=5)
result

# Playing with co-factor swap targets

In [ ]:
# Summary of some of the tested co-factor targets which had negative
# and positive influence on the biomass and squalene production

# NEGATIVE IMPROVEMENTS
#model.reactions.GLYCL
#model.reactions.ICDHyr
#model.reactions.DHDPRy
#model.reactions.SHK3Dr
#model.reactions.MDH
#model.reactions.ASAD

# POSITIVE IMPROVEMENTS
#model.reactions.GAPD
#model.reactions.FNOR
#model.reactions.EAR120y
#model.reactions.EAR60y

# Positive improvements

In [13]:
# Implement the jupyter notebook containing all positive co-factor swap targets 
%run CofSwap.ipynb 

# Check the swap effect

In [14]:
# Max biomass and squalene production after objective change
with model:
    swap_max_growth = model.slim_optimize()
    print("Max biomass growth:", swap_max_growth)
    model.objective = model.reactions.SQLS
    swap_max_production = model.slim_optimize()
    print("Max squalene production:", swap_max_production)

0.02254698645103219
0.02254698645103219


In [17]:
# Calcualte the percentage change in max biomass growth and squalene production
print("Change in max growth:", round((swap_max_growth-(base_max_growth))/(base_max_growth) * 100, 2), "%")
print("Change in max production:", round((swap_max_production-(base_max_production))/(base_max_production) * 100, 2), "%")

Change in max growth: 1.75 %
Change in max production: 1.75 %
